# Adversarial Attacks and Robustness ｜ FGSM

This demo demonstrates how to use the **fast gradient sign method  (FSGM)** [1] to generate adversarial examples. 

<img src="Util/demo2_1.jpg" alt="white-box attack" width="500"/>

[1] Goodfellow, Ian J., Jonathon Shlens, and Christian Szegedy. "Explaining and harnessing adversarial examples." ICLR 2015.

### Step 1: Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import matplotlib.pyplot as plt
import numpy as np

/Users/scarlettdu/CS/anacondasiyi/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/scarlettdu/CS/anacondasiyi/anaconda3/envs/py38/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/scarlettdu/CS/anacondasiyi/anaconda3/envs/py38/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <870081F6-12FD-3CEA-BC5C-30F4764F2A98> /Users/scarlettdu/CS/anacondasiyi/anaconda3/envs/py38/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <641FDF53-A831-3A7B-95BE-A5A95C6C5369> /Users/scarlettdu/CS/anacondasiyi/anaconda3/envs/py38/lib/python3.8/site-packages/torch/lib/libc10.dylib'If you don't plan on using image funct

ImportError: cannot import name 'QuantStub' from 'torch.ao.quantization' (/Users/scarlettdu/CS/anacondasiyi/anaconda3/envs/py38/lib/python3.8/site-packages/torch/ao/quantization/__init__.py)

### Step 2: Prepare the MNIST Dataset

In [2]:
# Transform for MNIST: Resize to 224x224 to match ResNet18 input
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# Load MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='Data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='Data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

AttributeError: module 'torch' has no attribute '_six'

### Step 3: Define the Model